In [5]:
import numpy as np
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from xgboost import XGBRegressor
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [6]:
nltk.download("stopwords")
nltk.download('vader_lexicon')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
# === 1. Loading the datasets ===
train_path = "/kaggle/input/beauty-products-review-rating-prediction/dataset/Train.csv"
test_path = "/kaggle/input/beauty-products-review-rating-prediction/dataset/Test.csv"
submission_path = "/kaggle/input/beauty-products-review-rating-prediction/dataset/Submission.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [8]:
# === 2. Cleaning the data ===
def clean_text(text):
    text = text.lower()  
    text = re.sub(r'\W+', ' ', text)  
    return text

train_df["clean_text"] = train_df["title"].astype(str) + " " + train_df["text"].astype(str)
train_df["clean_text"] = train_df["clean_text"].apply(clean_text)

test_df["clean_text"] = test_df["title"].astype(str) + " " + test_df["text"].astype(str)
test_df["clean_text"] = test_df["clean_text"].apply(clean_text)

In [9]:
# === 3. Embeddings from texts ===
device = "cuda" if torch.cuda.is_available() else "cpu"
sbert_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2").to(device)

train_embeddings = sbert_model.encode(
    train_df["clean_text"].tolist(), 
    batch_size=30,  
    show_progress_bar=True,
    convert_to_tensor=True,
    device=device
).half()  

test_embeddings = sbert_model.encode(
    test_df["clean_text"].tolist(),
    batch_size=30,
    show_progress_bar=True,
    convert_to_tensor=True,
    device=device
).half()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1333 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
# convert to DataFrame
train_embeddings_df = pd.DataFrame(train_embeddings.cpu().numpy())
test_embeddings_df = pd.DataFrame(test_embeddings.cpu().numpy())

In [11]:
# === 4.Sentiment Analysis ===
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def get_sentiment_score(text):
    return sia.polarity_scores(text)['compound']

train_df["sentiment_score"] = train_df["text"].astype(str).apply(get_sentiment_score)
test_df["sentiment_score"] = test_df["text"].astype(str).apply(get_sentiment_score)

train_embeddings_df["sentiment_score"] = train_df["sentiment_score"].values
test_embeddings_df["sentiment_score"] = test_df["sentiment_score"].values

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
# === 5. Categorical values encoding  ===
# Frequency Encoding for 'asin'
asin_counts = train_df["asin"].value_counts().to_dict()
train_df["asin_encoded"] = train_df["asin"].map(asin_counts)
test_df["asin_encoded"] = test_df["asin"].map(asin_counts).fillna(0)  # Заполняем NaN для теста

# Encoding 'verified_purchase'
train_df["verified_purchase"] = train_df["verified_purchase"].astype(int)
test_df["verified_purchase"] = test_df["verified_purchase"].astype(int)

train_embeddings_df["asin_encoded"] = train_df["asin_encoded"].values
train_embeddings_df["verified_purchase"] = train_df["verified_purchase"].values

test_embeddings_df["asin_encoded"] = test_df["asin_encoded"].values
test_embeddings_df["verified_purchase"] = test_df["verified_purchase"].values

In [13]:
train_df

,rating,title,text,asin,parent_asin,timestamp,helpful_vote,verified_purchase,clean_text,sentiment_score,asin_encoded
0,5,Unique and authentic,A very classy and chic look. This necklace is ...,B0107QYW14,B0107QYW14,1.460000e+12,2,1,unique and authentic a very classy and chic lo...,0.9013,88
1,2,Uncomfortable,The cloth bands tend to scrunch up under my wi...,B08WX159BW,B08WX159BW,1.630000e+12,0,1,uncomfortable the cloth bands tend to scrunch ...,0.2492,1
2,3,Something has changed for the worse,Really loved this product the first time I bou...,B083F76L79,B083F76L79,1.630000e+12,1,1,something has changed for the worse really lov...,-0.0781,1
3,5,Five Stars,beautiful,B00FBXJ11K,B00FBXJ11K,1.440000e+12,0,1,five stars beautiful,0.5994,2
4,3,1/4 and 3/4,I was so excited to get these. I opened the b...,B00F029PWC,B00F029PWC,1.520000e+12,0,1,1 4 and 3 4 i was so excited to get these i op...,0.9918,8
...,...,...,...,...,...,...,...,...,...,...,...
39981,5,Welcome relief,I have used DMSO for sometime and it works ver...,B0070Z7KME,B00J7QCNDU,1.420000e+12,0,1,welcome relief i have used dmso for sometime a...,0.2944,70
39982,3,Not sure yet.........,It holds basic hairstyles...but I am not in lo...,B01HCWXUIC,B01HCWXUIC,1.470000e+12,0,1,not sure yet it holds basic hairstyles but i a...,-0.2670,6
39983,5,*****5-STARS PLUS*****,Even though we all want to see immediate resul...,B01CYTUXHO,B01CYTUXHO,1.490000e+12,0,0,5 stars plus even though we all want to see i...,0.6638,8
39984,5,A fine shaver,Works very well and gives a nice shave.,B07K1DRCYR,B07K1DRCYR,1.560000e+12,0,1,a fine shaver works very well and gives a nice...,0.6361,20


In [14]:
train_df["text_length"] = train_df["text"].apply(len)
train_df["word_count"] = train_df["text"].apply(lambda x: len(x.split()))
test_df["text_length"] = test_df["text"].apply(len)
test_df["word_count"] = test_df["text"].apply(lambda x: len(x.split()))

In [15]:
train_embeddings_df["text_length"] = train_df["text_length"].values
train_embeddings_df["word_count"] = train_df["word_count"].values

test_embeddings_df["text_length"] = test_df["text_length"].values
test_embeddings_df["word_count"] = test_df["word_count"].values

train_embeddings_df.to_csv("/kaggle/working/train_embeddings_final.csv", index=False)
test_embeddings_df.to_csv("/kaggle/working/test_embeddings_final.csv", index=False)

np.save("/kaggle/working/train_embeddings_final.npy", train_embeddings_df.values)
np.save("/kaggle/working/test_embeddings_final.npy", test_embeddings_df.values)

In [16]:
# === 6. Preparing for training  ===
X_train = train_embeddings_df
X_test = test_embeddings_df
y_train = train_df["rating"].values

# === 7. Saving for simplicity  ===
X_train.to_csv("/kaggle/working/X_train.csv", index=False)
X_test.to_csv("/kaggle/working/X_test.csv", index=False)
np.save("/kaggle/working/y_train.npy", y_train)

In [17]:
X_train = pd.read_csv('/kaggle/working/X_train.csv')
X_test = pd.read_csv('/kaggle/working/X_test.csv')

In [18]:

# Split train into train (80%) and val (20%)
X_train, X_val, y_train_split, y_val = train_test_split(
    train_embeddings_df, y_train, test_size=0.2, random_state=42
)

In [19]:
X_train.columns = X_train.columns.astype(str)

from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy="auto", random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train_split)

print(f"Size of X_train: {X_train_balanced.shape}")

Size of X_train: (96410, 389)


In [20]:
X_train_balanced

,0,1,2,3,4,5,6,7,8,9,...,379,380,381,382,383,sentiment_score,asin_encoded,verified_purchase,text_length,word_count
0,-0.111572,0.061676,0.032623,-0.025894,0.073853,0.033630,0.048096,0.041962,0.010818,-0.018143,...,0.045654,-0.005592,0.030273,0.009789,0.050323,0.571900,46,1,17,2
1,-0.031189,-0.001369,0.038818,0.033447,-0.121948,-0.057068,0.043762,-0.019028,0.007591,0.068726,...,0.053131,0.033966,0.057983,0.040375,0.047668,0.764000,1,0,55,9
2,-0.064575,0.084656,0.055664,-0.069275,-0.110901,-0.015236,0.111938,-0.015762,-0.112427,-0.015991,...,-0.015945,-0.085938,-0.066406,-0.073853,0.056458,0.670500,2,1,133,23
3,-0.049683,0.047852,0.046570,-0.032745,-0.081238,0.046906,0.020447,-0.045898,-0.036316,0.067017,...,0.039948,-0.030090,-0.060577,0.023056,0.060059,0.497500,1,1,125,23
4,-0.101501,0.002840,0.021606,0.002195,-0.029282,-0.017670,0.123779,0.026657,0.044464,0.051483,...,0.067444,0.008636,-0.073792,-0.005600,0.007088,0.945900,1,1,128,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96405,-0.094177,0.066833,0.097168,0.002836,0.007458,-0.023407,0.097900,-0.045715,-0.048431,-0.041626,...,-0.042816,-0.023712,-0.062347,0.038879,0.048553,-0.271084,1,1,81,17
96406,-0.044922,0.059387,0.016113,0.001299,-0.045868,-0.014336,-0.056335,-0.045898,-0.036469,-0.015266,...,0.052338,0.011833,0.027618,0.048676,-0.008026,0.309128,1,1,37,6
96407,-0.047638,0.000610,0.053528,-0.033600,0.012390,0.007347,0.031250,0.035553,-0.047302,-0.039154,...,0.018295,0.024704,-0.049744,0.032990,0.061859,0.682123,114,1,630,115
96408,-0.016068,-0.005791,0.082275,0.006214,0.019333,-0.063293,0.035919,-0.040070,-0.044464,0.019241,...,0.030365,0.062561,-0.115784,-0.037445,0.004139,0.775932,1,0,157,33


# Training original sampled data

In [21]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


# XGBoost 
xgb_reg = XGBRegressor(n_estimators=300, max_depth=7, learning_rate=0.1, random_state=42)
xgb_reg.fit(X_train, y_train_split)

# RMSE
y_val_pred_xgb = xgb_reg.predict(X_val)
rmse_xgb = mean_squared_error(y_val, y_val_pred_xgb, squared=False)
print(f"RMSE XGBoost: {rmse_xgb:.4f}")

In [22]:
from catboost import CatBoostRegressor

#  CatBoost 
cat_reg = CatBoostRegressor(iterations=600, depth=7, learning_rate=0.1, verbose=100)
cat_reg.fit(X_train, y_train_split)

y_val_pred_cat = cat_reg.predict(X_val)
rmse_cat = mean_squared_error(y_val, y_val_pred_cat, squared=False)
print(f"RMSE CatBoost: {rmse_cat:.4f}")

In [23]:
from sklearn.ensemble import RandomForestRegressor

#  Random Forest
rf_reg = RandomForestRegressor(n_estimators=200, max_depth=6, random_state=42, verbose = 10)
rf_reg.fit(X_train, y_train_split)

y_val_pred_rf = rf_reg.predict(X_val)
rmse_rf = mean_squared_error(y_val, y_val_pred_rf, squared=False)
print(f"RMSE Random Forest: {rmse_rf:.4f}")

In [24]:
from sklearn.ensemble import VotingRegressor

# Ensembling
voting_reg = VotingRegressor(estimators=[
    ('xgb', xgb_reg),
    ('cat', cat_reg),
    ('rf', rf_reg)
])

voting_reg.fit(X_train, y_train_split)

# RMSE
y_val_pred_voting = voting_reg.predict(X_val)
rmse_voting = mean_squared_error(y_val, y_val_pred_voting, squared=False)
print(f"RMSE Voting Regressor: {rmse_voting:.4f}")


In [25]:
test_embeddings_df.columns = test_embeddings_df.columns.astype(str)

In [34]:
y_test_pred = voting_reg.predict(test_embeddings_df)

y_test_pred = np.clip(np.round(y_test_pred), 1, 5)

submission_path = "/kaggle/input/beauty-products-review-rating-prediction/dataset/Submission.csv"
submission_df = pd.read_csv(submission_path)

submission_df["rating"] = y_test_pred

submission_file = "/kaggle/working/submission_final_voting.csv"
submission_df.to_csv(submission_file, index=False)

print(f"✅ Финальный файл {submission_file} готов! 🚀")


✅ Финальный файл /kaggle/working/submission_final_voting.csv готов! 🚀


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  31 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  84 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  97 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 127 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 144 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Do

# Training with upsampled data

In [29]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

#  XGBoost
xgb_reg = XGBRegressor(n_estimators=300, max_depth=8, learning_rate=0.1, random_state=42)
xgb_reg.fit(X_train_balanced, y_train_balanced)

# #  CatBoost
cat_reg = CatBoostRegressor(iterations=500, depth=7, learning_rate=0.1, verbose=100)
cat_reg.fit(X_train_balanced, y_train_balanced)

#  Random Forest
rf_reg = RandomForestRegressor(n_estimators=60, max_depth=7, random_state=42, verbose = 10)
rf_reg.fit(X_train_balanced, y_train_balanced)

0:	learn: 1.3592057	total: 218ms	remaining: 1m 49s
100:	learn: 0.8443965	total: 15.9s	remaining: 1m 2s
200:	learn: 0.7659627	total: 31.3s	remaining: 46.5s
300:	learn: 0.7086089	total: 47.9s	remaining: 31.7s
400:	learn: 0.6619144	total: 1m 3s	remaining: 15.6s
499:	learn: 0.6213142	total: 1m 18s	remaining: 0us
building tree 1 of 60


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:   21.2s


building tree 2 of 60
building tree 3 of 60
building tree 4 of 60


[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:  1.4min


building tree 5 of 60
building tree 6 of 60
building tree 7 of 60


...

building tree 41 of 60
building tree 42 of 60
building tree 43 of 60
building tree 44 of 60
building tree 45 of 60
building tree 46 of 60
building tree 47 of 60
building tree 48 of 60
building tree 49 of 60


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed: 17.2min


building tree 50 of 60
building tree 51 of 60
building tree 52 of 60
building tree 53 of 60
building tree 54 of 60
building tree 55 of 60
building tree 56 of 60
building tree 57 of 60
building tree 58 of 60
building tree 59 of 60
building tree 60 of 60


[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed: 21.1min
[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed: 21.1min


RandomForestRegressor(max_depth=7, n_estimators=60, random_state=42, verbose=10)

In [34]:
X_train_balanced.columns = X_train_balanced.columns.astype(str)
X_val.columns = X_val.columns.astype(str)
test_embeddings_df.columns = test_embeddings_df.columns.astype(str)

y_val_pred_rf = xgb_reg.predict(X_val)
rmse_rf = mean_squared_error(y_val, y_val_pred_rf, squared=False)
print(f"RMSE Random Forest: {rmse_rf:.4f}")

RMSE Random Forest: 0.9206


In [35]:
from sklearn.ensemble import VotingRegressor

voting_reg = VotingRegressor(estimators=[
    ('xgb', xgb_reg),
    ('cat', cat_reg),
    ('rf', rf_reg)
])

voting_reg.fit(X_train_balanced, y_train_balanced)

y_val_pred_voting = voting_reg.predict(X_val)
rmse_voting = mean_squared_error(y_val, y_val_pred_voting, squared=False)
print(f"RMSE Voting Regressor) balanced: {rmse_voting:.4f}")

0:	learn: 1.3592057	total: 219ms	remaining: 1m 49s
100:	learn: 0.8443965	total: 15.9s	remaining: 1m 2s
200:	learn: 0.7659627	total: 30.6s	remaining: 45.5s
300:	learn: 0.7086089	total: 45.8s	remaining: 30.3s
400:	learn: 0.6619144	total: 1m	remaining: 14.9s
499:	learn: 0.6213142	total: 1m 15s	remaining: 0us
building tree 1 of 60


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:   21.3s


building tree 2 of 60
building tree 3 of 60
building tree 4 of 60


[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:  1.4min


building tree 5 of 60
building tree 6 of 60
building tree 7 of 60


...

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed: 17.3min


building tree 50 of 60
building tree 51 of 60
building tree 52 of 60
building tree 53 of 60
building tree 54 of 60
building tree 55 of 60
building tree 56 of 60
building tree 57 of 60
building tree 58 of 60
building tree 59 of 60
building tree 60 of 60


[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed: 21.2min
[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed: 21.2min


🔹 RMSE Ансамбля (Voting Regressor) с балансировкой: 0.9582


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  31 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed:    0.1s


In [36]:
y_test_pred = voting_reg.predict(test_embeddings_df)
y_test_pred = np.clip(np.round(y_test_pred), 1, 5)

submission_path = "/kaggle/input/beauty-products-review-rating-prediction/dataset/Submission.csv"
submission_df = pd.read_csv(submission_path)

submission_df["rating"] = y_test_pred

submission_file = "/kaggle/working/submission_final_balanced_voting.csv"
submission_df.to_csv(submission_file, index=False)

✅ Финальный файл /kaggle/working/submission_final_balanced_voting.csv готов! 🚀


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  31 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed:    0.0s
